In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('clientes_sem_outliers.csv')
df.head()

,nome,cpf,idade,data,endereco,estado
0,Sr. Vitor Gabriel Silva,640.719.385-01,90,1933-12-06,"Trevo Emanuel Silva, 482, Vila Tirol, 47847085...",Rondônia
1,Calebe Santos,651.297.384-46,40,1984-03-06,"Recanto Moraes, 7, Ernesto Nascimento, 96497-6...",Rio de Janeiro
2,Evelyn Pires,107.643.528-90,80,1944-03-09,"Travessa de Caldeira, 237, Santa Terezinha, 84...",Sergipe
3,Maria Julia Castro,164.750.293-43,76,1947-04-12,"Residencial de Pereira, 20, Jatobá, 62127-165 ...",Pernambuco
4,Arthur Da Rosa,085.167.249-30,53,1970-04-16,"Parque Almeida, 66, Vila Suzana Segunda Seção,...",Santa Catarina
